# Collecting data from ThermoML xml files

In [1]:
!python setup.py build_ext --inplace

Compiling ThermoMLBuilder.pyx because it changed.
[1/1] Cythonizing ThermoMLBuilder.pyx
/usr/local/lib/python3.9/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /workspaces/ThermoMLDataCollector/ThermoMLBuilder.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'ThermoMLBuilder' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/include/python3.9 -c ThermoMLBuilder.c -o build/temp.linux-x86_64-cpython-39/ThermoMLBuilder.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 build/temp.linux-x86_64-cpython-39/ThermoMLBuilder.o -L/usr/lib/x86_64-linux-gnu -o build/lib.linux-x86_64-cpython-39/ThermoMLBuilder.cpython-39-x86_64-linux-gnu.so
copyi

In [2]:
# pyxbgen -u ThermoML/ThermoML.xsd -m ThermoML/thermoml_schema

In [3]:
import os
from ThermoMLBuilder import build_dataset
import polars as pl
import numpy as np
from importlib import reload
import glob
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt

thermoml_schema.py generator:

In [4]:
def RetrieveFiles():
    """Outputs a list of files path from reference DOIs (taken from https://trc.nist.gov/ThermoML/Browse)
    
    Parameters
    ----------
    ref: str
        file path for the file with the reference DOIs

    Outputs
    ----------
    files: list(str)
        list of paths for each reference file
    """


    dir = os.getcwd()
    files = [os.path.join(dir, file)
                for file in glob.glob('ThermoML/*/*.xml')]
    return files


## Raw data

Collecting data from all files

In [5]:
files = RetrieveFiles() 
files[0]

'/workspaces/ThermoMLDataCollector/ThermoML/10.1007/s10765-005-5566-6.xml'

In [6]:
data = build_dataset(files,'data','raw')

files:   1%|          | 121/11923 [01:39<3:04:07,  1.07it/s]

In [ ]:
df = pl.read_parquet('../ePC-SAFT/data/thermoml/raw/dataden.parquet')

In [ ]:
def mw(inchi):
    
    try:
        mol = Chem.MolFromInchi(inchi, removeHs=False)
        mol = Chem.AddHs(mol)
        mol_weight = CalcExactMolWt(mol)
    except:
        mol_weight = 0

    return mol_weight

In [ ]:
mw1 = df['inchi1'].apply(mw).rename('mw1')

In [ ]:
df = df.with_columns(mw1)

In [ ]:
mw2 = df['inchi2'].apply(mw).rename('mw2')

In [ ]:
df = df.with_columns(mw2)

In [ ]:
df.write_parquet('../ePC-SAFT/data/thermoml/raw/dataden.parquet')